# Neural Network for Density Forecasting EC331

In [ ]:
#######################################################################
#######################################################################
######################### Importing Packages ##########################
#######################################################################
#######################################################################

# plotting packages
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# numpy and pandas
import numpy as np
from numpy.random import rand
from numpy import ix_
import pandas as pd

# miscellany to make code neater
from typing import Callable
import math
from datetime import datetime, timedelta

# some basic statistical/numerical scipy parts used to calculate scores
import scipy.stats
import scipy.integrate as integrate
import scipy.special as special

# Tensorflow and Keras parts which are used for data processing and model creation
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# tensorflow pdf calculator which is used to calculate losses efficiently
import tensorflow_probability as tfp
tfd = tfp.distributions

# setting seed as stochastic intialisation
tf.random.set_seed(11)
np.random.seed(1337)

# Data Importing and Pre-Processing

In [ ]:
#######################################################################
#######################################################################
###################### Data Importing/Cleaning ########################
#######################################################################
#######################################################################

main_path = "C:/Warwick Final Year/RAE/"
raw_path = main_path + "Data/"
processed_path = main_path + "Processed Data/"
graphs_path = main_path + "Graphs/"
checkpoints_path = main_path + "Code Python/Model Checkpoints/"

# import data and calculate log returns from adjusted close
df_Nikkei_RAW = pd.read_csv(raw_path + "^N225.csv")
df_NASDAQ_RAW = pd.read_csv(raw_path + "^IXIC.csv")
df_DAX_RAW = pd.read_csv(raw_path + "^GDAXI.csv")

In [ ]:
def Data_Processor(DATA: pd.DataFrame,
                   batch_size: int,
                   length_sample: int,
                   test_train_split: str) -> dict:
    
    """
    #################################################################################################################
    #################################################################################################################
    
    Processes data to a usable form. Splits data into a training and test set based on date
    given by 'test_train_split'. Makes data automatically batch and use a lag length of 'length_sample'.
    
    #################################################################################################################
    #################################################################################################################
    
    @param DATA: pd.DataFrame, stock index data from Yahoo Finance with Adjusted Closing prices and Date as columns
    @param batch_size: int, size of batch used whilst training models
    @param length_sample: int, number of lagged trading days to use whilst forecasting each timestep
    @param test_train_split: str, date at which to split data for training and validation - format of "dd-mm-yyyy"
    
    #################################################################################################################
    #################################################################################################################
    
    @return: dict, dictionary with 3 keys. DATA key is the log return data for the given series after cleaning
                                           Training key is the training data which is batched and has given sample size
                                           Validation key is the validation data which is batch and has given sample size
    
    #################################################################################################################
    #################################################################################################################
    """
    
    DATA.columns = [c.replace(' ', '_') for c in DATA.columns]
    DATA = DATA[DATA['Adj_Close'].notnull()]
    DATA['log_ret'] = np.log(DATA.Adj_Close) - np.log(DATA.Adj_Close.shift(1))

    # spilt to training and test sets
    DATA    = DATA[['Date', 'log_ret']][1:]
    DATA['Date'] = DATA['Date'].apply(pd.Timestamp)
    DATA.set_index('Date', inplace=True, drop=True)
    
    train = DATA.loc[:test_train_split]
    test  = DATA.loc[test_train_split:]
   
    DATA_train = [[i] for i in train['log_ret']]
    DATA_test  = [[i] for i in test['log_ret']]
    
    time_series_generator = TimeseriesGenerator(DATA_train, 
                                                DATA_train, 
                                                length = length_sample, 
                                                batch_size = batch_size)
    time_series_val_generator = TimeseriesGenerator(DATA_test,
                                                    DATA_test, 
                                                    length = length_sample, 
                                                    batch_size = batch_size)

    
    return {"Data": DATA,
            "Training": time_series_generator,
            "Validation": time_series_val_generator}

In [ ]:
# now lead in the Nikkei, NASDAQ, and DAX 

Batch_Size = 64
Length = 10
Test_Train_Split = '2015-01-01'

Nikkei = Data_Processor(df_Nikkei_RAW,
                       Batch_Size,
                       Length,
                       Test_Train_Split)
NASDAQ = Data_Processor(df_NASDAQ_RAW,
                       Batch_Size,
                       Length,
                       Test_Train_Split)
DAX = Data_Processor(df_DAX_RAW,
                       Batch_Size,
                       Length,
                       Test_Train_Split)

DATA = {
       "Nikkei": Nikkei,
       "NASDAQ": NASDAQ,
       "DAX": DAX
}

# Plotting Stock Returns

In [ ]:
for count,i in enumerate([Nikkei['Data'],
         NASDAQ['Data'],
         DAX['Data']]):
    

    if count == 0:
        NAME = "Nikkei"
    elif count == 1:
        NAME = "NASDAQ"
    else:
        NAME = "DAX"
    
    ax = i.plot(y = 'log_ret', 
            kind = 'line',
            rot = 45,
            legend = False,
            title = NAME
           )
    
    
    means = np.repeat(0, len(i['log_ret']))
    std   = np.repeat(np.std(i['log_ret']), len(i['log_ret']))
    x     = pd.date_range(str(i.index.values[0]), str(i.index.values[-1]), freq = "D")
    ax.axvline(pd.to_datetime('2015-01-01'), color = 'r', linestyle = '--')
    
    s1 = ax.fill_between(i.index.values, 
                         np.add(means,std),
                         np.subtract(means,std), 
                         color = 'green', 
                         zorder = 4, 
                         alpha = 0.4)
    
    s2 = ax.fill_between(i.index.values, 
                         np.add(means,np.multiply(2,std)),
                         np.subtract(means,np.multiply(2,std)), 
                         color = 'grey',
                         zorder = 3,
                         alpha = 0.5)
    
    ax.text(pd.to_datetime('2016-06-01'), 
            -0.18,
        "Test/Train Split",
        horizontalalignment = 'center', 
            fontweight = 'bold', 
            color = 'red', 
            rotation = -90,
           fontsize = 'x-small')
    
    years = mdates.YearLocator(10)   # every year
    years_fmt = mdates.DateFormatter('%Y')
    
    
    ax.xaxis.set_major_locator(years)
    ax.xaxis.set_major_formatter(years_fmt)
    ax.set(ylabel = "Log Returns")
    ax.legend(handles = [s1,s2], labels = ["1 std","2 std"], loc = 'upper left')
    ax.set_ylim([-0.2, 0.2])
    plt.tight_layout() 
    ax.figure.savefig(graphs_path + str(count) + 'logret.pdf')

# The Neural Network Architecture

These neural network models work by assuming some parametrised distribution for the stock returns and forecasting the parameters of the distribution in the following period. 
\
As a loss function we use the log of the probability density function. Thus, our neural network is in effect being trained to converge to the maximum likelihood estimators. 
\
Basic building block is LSTM layers interspersed with dropout layers for regularisation.

In [ ]:
class Model(tf.keras.Model):
    
    def __init__(self, distribution):
        super().__init__()
        
        self.distribution = distribution
        
        # conditional mean channel
        self.LSTM1 = tf.keras.layers.LSTM(16, 
                                          activation = tf.nn.relu,
                                          return_sequences=True)
        
        self.LSTM2 = tf.keras.layers.LSTM(32, 
                                          activation = tf.nn.relu,
                                          return_sequences=True)
        
        self.LSTM3 = tf.keras.layers.LSTM(16, 
                                          activation = tf.nn.relu,
                                          return_sequences=False)
        
        
        # scale/std channel 
        self.LSTM4 = tf.keras.layers.LSTM(16, 
                                          activation = tf.nn.relu,
                                          return_sequences=True)
        
        self.LSTM5 = tf.keras.layers.LSTM(32, 
                                          activation = tf.nn.relu,
                                          return_sequences = True)
        
        
        self.LSTM6 = tf.keras.layers.LSTM(16, 
                                          activation = tf.nn.relu,
                                          return_sequences = False)
        
        # DoF Channel
        
        self.LSTM7 = tf.keras.layers.LSTM(16, 
                                          activation = tf.nn.relu,
                                          return_sequences = True)
        
        self.LSTM8 = tf.keras.layers.LSTM(32, 
                                          activation = tf.nn.relu,
                                          return_sequences = True)
        
        
        self.LSTM9 = tf.keras.layers.LSTM(16, 
                                          activation = tf.nn.relu,
                                          return_sequences = False)


        # dropout layers to regularise
        self.dropout1 = tf.keras.layers.Dropout(0.3)
        self.dropout2 = tf.keras.layers.Dropout(0.4)
        self.dropout3 = tf.keras.layers.Dropout(0.3)
        
        self.dropout4 = tf.keras.layers.Dropout(0.3)
        self.dropout5 = tf.keras.layers.Dropout(0.4)
        self.dropout6 = tf.keras.layers.Dropout(0.3)
        
        self.dropout7 = tf.keras.layers.Dropout(0.3)
        self.dropout8 = tf.keras.layers.Dropout(0.4)
        self.dropout9 = tf.keras.layers.Dropout(0.3)



        # a dense layer for conditional mean
        self.dense1 = tf.keras.layers.Dense(1, 
                                            activation='tanh')
        
        # another for the scale parameter
        self.dense2 = tf.keras.layers.Dense(1, 
                                            activation='sigmoid')
        
        # for t-distribution the scale parameter doesn't correspond to std 
        # DoF >0 and scale >0 so use relu for these
        self.dense3 = tf.keras.layers.Dense(1,
                                           activation='relu')
        
        self.dense4 = tf.keras.layers.Dense(1,
                                           activation='relu')
        
    

    def call(self, inputs):
        # LSTM --> Dropout --> dense with 2 outputs (conditional mean & std)
        
        if self.distribution in ['Normal',"Laplace"] :
        
            # mean channel
            x1 = self.LSTM1(inputs)
            x1 = self.dropout1(x1)    
            x1 = self.LSTM2(x1)
            x1 = self.dropout2(x1)
            x1 = self.LSTM3(x1)
            x1 = self.dropout3(x1)

            # std channel
            x2 = self.LSTM4(inputs)
            x2 = self.dropout4(x2)
            x2 = self.LSTM5(x2)
            x2 = self.dropout5(x2)
            x2 = self.LSTM6(x2)
            x2 = self.dropout6(x2)

            out1 = self.dense1(x1)
            out2 = self.dense2(x2)

            return concatenate([out1,out2])
        
        elif self.distribution == 't':
            
             # mean channel
            x1 = self.LSTM1(inputs)
            x1 = self.dropout1(x1)    
            x1 = self.LSTM2(x1)
            x1 = self.dropout2(x1)
            x1 = self.LSTM3(x1)
            x1 = self.dropout3(x1)

            # scale channel
            x2 = self.LSTM4(inputs)
            x2 = self.dropout4(x2)
            x2 = self.LSTM5(x2)
            x2 = self.dropout5(x2)
            x2 = self.LSTM6(x2)
            x2 = self.dropout6(x2)
            
            # DoF channel for t dist
            
            x3 = self.LSTM7(inputs)
            x3 = self.dropout7(x3)
            x3 = self.LSTM8(x3)
            x3 = self.dropout8(x3)
            x3 = self.LSTM9(x3)
            x3 = self.dropout9(x3)

            out1 = self.dense1(x1)
            out2 = self.dense3(x2)
            out3 = self.dense4(x3)

            return concatenate([out1,out2,out3])
            
    


In [ ]:
def loss_func(distribution: str) -> Callable[[list,list], float]:
    
    """
    #################################################################################################################
    #################################################################################################################
    
    Generates the loss function for the specified distribution. Adds small numbers (1e-10) to some parameters
    so that learning doesn't produce NaN accidentally and stop learning. There is a way to integrate this
    with the above class but it keeps throwing errors when I try so forgive the sloppy code!
    
    #################################################################################################################
    #################################################################################################################
    
    @param distribution: str, the chosen distribution to generate the loss function for
    
    #################################################################################################################
    #################################################################################################################
    
    @return loss_comp: loss function taking in the y_true target variable and the predicted y_pred variables
                       and outputs the average loss over the batch
                       
    #################################################################################################################
    #################################################################################################################
    """
    
    
    if distribution == "Normal":
        def loss_comp(y_true, y_pred):
            (mean_pred, scale_pred) = tf.split(y_pred, num_or_size_splits=2, axis=1)
            dist = tfd.Normal(loc = mean_pred, scale=tf.math.add(scale_pred,1e-10))   
            log_dens = tf.math.log(tf.math.add(dist.prob(y_true),1e-10))
            loss = -tf.math.reduce_mean(log_dens)
            return loss
            
    elif distribution == "t":
        def loss_comp(y_true, y_pred):
            (mean_pred, scale_pred, nu_pred) = tf.split(y_pred, num_or_size_splits=3, axis=1)
            dist = tfd.StudentT(df = tf.math.add(nu_pred,1e-10), loc = mean_pred, scale=tf.math.add(scale_pred,1e-10))   
            log_dens = tf.math.log(dist.prob(y_true))
            loss = -tf.math.reduce_mean(log_dens)
            
            return loss
        
    elif distribution == "Laplace":
        def loss_comp(y_true, y_pred):
            (mean_pred, scale_pred) = tf.split(y_pred, num_or_size_splits=2, axis=1)
            dist = tfd.Laplace(loc = mean_pred, 
                               scale=tf.math.add(scale_pred,1e-10))   
            log_dens = tf.math.log(tf.math.add(dist.prob(y_true),1e-10))
            loss = -tf.math.reduce_mean(log_dens)
            return loss
        
    return loss_comp
                     

In [ ]:
# create callbacks
callbacks = {}
for i in ["Normal","t","Laplace"]:

    callbacks[i] = [ ModelCheckpoint(filepath=checkpoints_path + i,
                                         monitor='val_loss',
                                         verbose=1,
                                         save_weights_only=True,
                                         save_best_only=True),
                    ReduceLROnPlateau(monitor='val_loss',
                                              factor=0.1,
                                              min_lr=1e-10,
                                              patience=10,
                                              verbose=1),
                   EarlyStopping(monitor='loss', patience=50)]



# Training the Models

In [ ]:
# Pick which series to fit:

which_series = "Nikkei"

if which_series not in ["Nikkei","NASDAQ","DAX"]:
    raise Exception("That series is not available")

In [ ]:
time_series_generator = DATA[which_series]["Training"]
time_series_val_generator = DATA[which_series]['Validation']

In [ ]:
# Select Hyperparameters for training

num_epochs = 2000

learning_rates = {
                  "t"       : 0.0001,
                  "Normal"  : 0.01,
                  "Laplace" : 0.001
}

loss_functions = {
                  "t"       : loss_func("t"),
                  "Normal"  : loss_func("Normal"),
                  "Laplace" : loss_func("Laplace")
}

In [ ]:
# first we fit the model using a t-distribution
model_t = Model("t")

model_t.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rates["t"]),
                loss=loss_functions["t"])


t_History = model_t.fit(time_series_generator,        
                        epochs = num_epochs, 
                        shuffle = False,
                        validation_data=time_series_val_generator,
                        callbacks=callbacks["t"])
             

In [ ]:
# Next we fit the model with a normal distribution

model_Normal = Model("Normal")

model_Normal.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rates["Normal"]),
                     loss=loss_functions["Normal"])


Normal_History = model_Normal.fit(time_series_generator,        
                                  epochs = num_epochs, 
                                  shuffle = False,
                                  validation_data=time_series_val_generator,
                                  callbacks=callbacks["Normal"])

In [ ]:
# Finally fit the model with a Laplace distribution

model_Laplace = Model("Laplace")

model_Laplace.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rates["Laplace"]),
                      loss=loss_functions["Laplace"])

Laplace_History = model_Laplace.fit(time_series_generator,        
                                    epochs = num_epochs, 
                                    shuffle = False,
                                    validation_data=time_series_val_generator,
                                    callbacks=callbacks["Laplace"])

# Plotting Forecasts
Plot 5% and 10% VaR bands for each forecasted distribution and compared to observed stock returns.

In [ ]:
Val_Pred_t     = model_t.predict(DATA[which_series]['Validation'])
Val_Pred_Normal = model_Normal.predict(DATA[which_series]['Validation'])
Val_Pred_Laplace = model_Laplace.predict(DATA[which_series]['Validation'])

In [ ]:
# validation set is from 2015-01-01, also need length of window forecasting over to predict first data point
validation_data = DATA[which_series]["Data"].loc['2015-01-01':]['log_ret'][Length:]

In [ ]:
# first plot Normal
means_N = [i[0] for i in Val_Pred_Normal]
std_N   = [i[1] for i in Val_Pred_Normal]
x = pd.date_range("2015-01-01", periods=len(means_N),freq="D")


plt.plot(x,validation_data, color='#1f77b4',zorder=1)

ppf = scipy.stats.norm.ppf
SCORE = -round(Normal_History.history["val_loss"][-1],4)
NAME = "Normal"
s1 = plt.fill_between(x,np.add(means_N,ppf(0.95,loc=means_N,scale=std_N)),np.add(means_N,ppf(0.05,loc=means_N,scale=std_N)), 
                      color='green',zorder=4,alpha=0.4)
s2 = plt.fill_between(x,np.add(means_N,ppf(0.975,loc=means_N,scale=std_N)),
                 np.add(means_N,ppf(0.025,loc=means_N,scale=std_N)), 
                 color='grey',
                     zorder=3,
                     alpha=0.5)
plt.plot(x,means_N, color='black',zorder=2)

plt.title(NAME)

plt.text(pd.to_datetime('2019-06-01'), 
        -0.08,
    "SCORE="+str(SCORE),
    horizontalalignment='center', 
        fontweight='bold', 
        color='red',
       fontsize='medium')

years = mdates.YearLocator(10)  
years_fmt = mdates.DateFormatter('%Y')


plt.ylabel("Log Returns")
plt.legend(handles=[s1,s2], labels=["10%","5%"],loc='upper right')
plt.ylim([-0.15, 0.15])
plt.tight_layout() 

plt.savefig(graphs_path + which_series + '_Validation_' + NAME + '.pdf')

plt.show()

In [ ]:
# Now t
means_T = [i[0] for i in Val_Pred_t]
scale_T   = [i[1] for i in Val_Pred_t]
dfs_T   = [i[2] for i in Val_Pred_t]
x = pd.date_range("2015-01-01", periods=len(means_T),freq="D")


plt.plot(x,validation_data, color='#1f77b4',zorder=1)

ppf = scipy.stats.t.ppf
         
SCORE = -round(t_History.history["val_loss"][-1],4)
NAME = "Student_T"
         
s1 = plt.fill_between(x,np.add(means_T,ppf(0.95,df=dfs_T, loc=means_T,scale=scale_T)),
                      np.add(means_T,ppf(0.05,df=dfs_T,loc=means_T,scale=scale_T)), 
                      color='green',zorder=4,alpha=0.4)
s2 = plt.fill_between(x,np.add(means_T,ppf(0.975,df=dfs_T,loc=means_T,scale=scale_T)),
                 np.add(means_T,ppf(0.025,df=dfs_T,loc=means_T,scale=scale_T)), 
                 color='grey',
                     zorder=3,
                     alpha=0.5)
plt.plot(x,means_T, color='black',zorder=2)

plt.title(NAME)

plt.text(pd.to_datetime('2019-06-01'), 
        -0.08,
    "SCORE="+str(SCORE),
    horizontalalignment='center', 
        fontweight='bold', 
        color='red',
       fontsize='medium')

years = mdates.YearLocator(10)  
years_fmt = mdates.DateFormatter('%Y')

plt.ylabel("Log Returns")
plt.legend(handles=[s1,s2], labels=["10%","5%"],loc='upper right')
plt.ylim([-0.15, 0.15])
plt.tight_layout() 

plt.savefig(graphs_path + which_series + '_Validation_' + 'Student_T' + '.pdf')

plt.show()

In [ ]:
# Lastly the Laplacian
means_L = [i[0] for i in Val_Pred_Laplace]
scale_L   = [i[1] for i in Val_Pred_Laplace]
x = pd.date_range("2015-01-01", periods=len(means_L),freq="D")


plt.plot(x,validation_data, color='#1f77b4',zorder=1)

ppf = scipy.stats.laplace.ppf
         
SCORE = -round(Laplace_History.history["val_loss"][-1],4)
NAME = "Laplace"
         
s1 = plt.fill_between(x,np.add(means_L,ppf(0.95,loc=means_L,scale=scale_L)),
                      np.add(means_L,ppf(0.05,loc=means_L,scale=scale_L)), 
                      color='green',zorder=4,alpha=0.4)
s2 = plt.fill_between(x,np.add(means_L,ppf(0.975,loc=means_L,scale=scale_L)),
                 np.add(means_L,ppf(0.025,loc=means_L,scale=scale_L)), 
                 color='grey',
                     zorder=3,
                     alpha=0.5)
plt.plot(x,means_L, color='black',zorder=2)

plt.title(NAME)

plt.text(pd.to_datetime('2019-06-01'), 
        -0.08,
    "SCORE="+str(SCORE),
    horizontalalignment='center', 
        fontweight='bold', 
        color='red',
       fontsize='medium')

years = mdates.YearLocator(10)  
years_fmt = mdates.DateFormatter('%Y')

plt.ylabel("Log Returns")
plt.legend(handles=[s1,s2], labels=["10%","5%"],loc='upper right')
plt.ylim([-0.15, 0.15])
plt.tight_layout() 

plt.savefig(graphs_path + which_series + '_Validation_' + NAME + '.pdf')

plt.show()

# Exporting losses and predictions

In [ ]:
# helpers 
Normal_pdfs = scipy.stats.norm.pdf(
                                   x=DAX['Data'].loc['2015-01-01':]['log_ret'][10:],
                                   loc=means_N,
                                   scale=std_N
                                )

T_pdfs = scipy.stats.t.pdf(
                                   x=DAX['Data'].loc['2015-01-01':]['log_ret'][10:],
                                   df=dfs_T,
                                   loc=means_T,
                                   scale=scale_T
                                )

Laplace_pdfs = scipy.stats.laplace.pdf(
                                   x=DAX['Data'].loc['2015-01-01':]['log_ret'][10:],
                                   loc=means_L,
                                   scale=scale_L
                                )

Normal_L2 =[0 for i in range(len(means_N))]
T_L2 = [0 for i in range(len(means_N))]
Laplace_L2 =[0 for i in range(len(means_N))]

for i in range(len(means_N)):
    
    Normal_L2[i] = np.power(integrate.quad(lambda x: np.power(scipy.stats.norm.pdf(x,
                                                                              loc=means_N[i],
                                                                              scale=std_N[i]),2),-np.inf,np.inf)[0] , 0.5)
    
    T_L2[i] = np.power(integrate.quad(lambda x: np.power(scipy.stats.t.pdf(x,
                                                                      df=dfs_T[i],
                                                                      loc=means_T[i],
                                                                      scale=scale_T[i]),2),-np.inf,np.inf)[0] , 0.5)
    
    Laplace_L2[i] = np.power(integrate.quad(lambda x: np.power(scipy.stats.laplace.pdf(x,
                                                                               loc=means_L[i],
                                                                               scale=scale_L[i]),2),-np.inf,np.inf)[0] , 0.5)


# get the series of validation log losses for the three models

T_losses_log       =  -np.log(T_pdfs
                         )

Normal_losses_log  = -np.log(Normal_pdfs
                        )

Laplace_losses_log = -np.log(Laplace_pdfs
                        )

# also the quad losses

Normal_losses_Quad = -(np.multiply(Normal_pdfs,2)-
                                  np.power(Normal_L2,2))
            
Laplace_losses_Quad = -(np.multiply(Laplace_pdfs,2)-
                      np.power(Laplace_L2,2))


T_losses_Quad = -(np.multiply(T_pdfs,2)-
                      np.power(T_L2,2))
#finally the spherical scores

Normal_losses_Sph = -np.divide(Normal_pdfs,Normal_L2)

Laplace_losses_Sph = -np.divide(Laplace_pdfs,Laplace_L2)

T_losses_Sph = -np.divide(T_pdfs,T_L2)

In [ ]:
d = {
    'Student_T_losses_log' : T_losses_log,
    'Normal_losses_log'    : Normal_losses_log,
    'Laplace_losses_log'   : Laplace_losses_log,
    'Student_T_losses_quad' : T_losses_Quad,
    'Normal_losses_quad'    : Normal_losses_Quad,
    'Laplace_losses_quad'   : Laplace_losses_Quad,
    'Student_T_losses_sph' : T_losses_Sph,
    'Normal_losses_sph'    : Normal_losses_Sph,
    'Laplace_losses_sph'   : Laplace_losses_Sph,
    'Normal_loc'       : means_N,
    'Normal_scale'     : std_N,
    'Student_T_loc'    : means_T,
    'Student_T_scale'  : scale_T,
    'Student_T_shape'  : dfs_T,
    'Laplace_loc'      : means_L,
    'Laplace_scale'    : scale_L
    }

df = pd.DataFrame(data=d)

In [ ]:
df.to_csv(processed_path + which_series + '_predictions_losses.csv')